In [ ]:
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START
from langgraph.graph import MessagesState

class State(MessagesState):
    summary: str
# Define a new graph


In [ ]:
workflow = StateGraph(State)
workflow.add_node("Query", check_ambiguity)
workflow.add_node("Query Rewrite", rewrite_query)
workflow.add_node("Context Augmentation", augment_query_with_context)
workflow.add_node("Question Clarification", clarify_question)
workflow.add_node("Ask User", ask_user)
workflow.add_node("Response", respond)
workflow.add_node("Conversation check", check_conversation_length)
workflow.add_node("Summarization", summarize_conversation)
workflow.add_node("Session memory store", store_session_memory)


workflow.set_entry_point("Query")
workflow.add_conditional_edges("Query", is_ambiguous, {"ambiguous": "Query Rewrite", "not_ambiguous": "Context Augmentation"})
workflow.add_edge("Query Rewrite", "Context Augmentation")
workflow.add_edge("Context Augmentation", "Question Clarification")
workflow.add_conditional_edges("Question Clarification", is_clarified, {"clarified": "Response", "not_clarified": "Ask User"})
workflow.add_edge("Ask User", END)
workflow.add_edge("Response", "Conversation check")
workflow.add_conditional_edges("Conversation check", should_summarize, {"summarize": "Summarization", "continue": END})
workflow.add_edge("Summarization", "Session memory store")
workflow.add_edge("Session memory store", END)

In [ ]:
graph = workflow.compile()